In [ ]:
library("sva")
library("ggplot2")
library("gridExtra")
library("stringr")

In [ ]:
FILEs=list('GeneCount_mfa.txt','GeneCPM_mfa.txt','TECount_filled_mfa.txt','GeneTPM_TIMER_sample_mfa.txt','TEtranscript_all.txt','miRNACount_mfa.txt')

for (i in FILEs){
    U_readcount <- read.table(paste0('./source_data/count_result/',i),header=T, sep="\t",as.is=c(1))
    colnames(U_readcount)[1]<-'Gene'
    split_result = str_split(U_readcount$Gene,"_", 3)
    U_readcount$Chr = lapply(X = split_result, FUN = "[", 2)
    U_readcount <- U_readcount[,c(1,26,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25)]

    names(U_readcount) = c("Gene", "Chr",
                           "G1_Year1_1", "G1_Year1_2", "G1_Year2_3", "G1_Year2_4", "G1_Year3_5", "G1_Year3_6",
                           "G2_Year1_1", "G2_Year1_2", "G2_Year2_3", "G2_Year2_4", "G2_Year3_5", "G2_Year3_6",
                           "G3_Year1_1", "G3_Year1_2", "G3_Year2_3", "G3_Year2_4", "G3_Year3_5", "G3_Year3_6",
                           "G4_Year1_1", "G4_Year1_2", "G4_Year2_3", "G4_Year2_4", "G4_Year3_5", "G4_Year3_6")

    sample_names = names(U_readcount)[3:length(names(U_readcount))]

    Age_group = c("G1", "G1", "G1", "G1", "G1", "G1", 
                   "G2", "G2", "G2", "G2", "G2", "G2", 
                   "G3", "G3", "G3", "G3", "G3", "G3", 
                   "G4", "G4", "G4", "G4", "G4", "G4")
    Year_sampling = c("Year1", "Year1", "Year2", "Year2", "Year3", "Year3", 
                        "Year1", "Year1", "Year2", "Year2", "Year3", "Year3", 
                        "Year1", "Year1", "Year2", "Year2", "Year3", "Year3", 
                        "Year1", "Year1", "Year2", "Year2", "Year3", "Year3")
    replicates = c(1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6)

    pca_uncorrected_obj = prcomp(U_readcount[,sample_names])

    pca_uncorrected = as.data.frame(pca_uncorrected_obj[2]$rotation)

    pca_uncorrected[,"Age_group"] = Age_group
    pca_uncorrected[,"Year_sampling"] = Year_sampling
    pca_uncorrected[,"replicate"] = replicates

    p1 = ggplot(data=pca_uncorrected, aes(x=PC1, y=PC2, color=Age_group, shape=Year_sampling))
    p1 = p1 + stat_ellipse(type="norm", linetype=2)
    p1 = p1 + theme_bw() + theme(axis.title = element_text(size=20),panel.border = element_rect(size = 2, fill = NA),axis.text.y= element_text(size = 15),axis.text.x= element_text(size = 15))+
    geom_point(aes(color=Age_group), size =5)


    #perform the batch correction
    groups = sapply(as.character(Age_group), switch, "G1" = 1, "G2" = 2, "G3" = 3, "G4" = 4, USE.NAMES = F)
    batches = sapply(as.character(Year_sampling), switch, "Year1" = 1, "Year2" = 2, "Year3" = 3, USE.NAMES = F)

    #now run ComBat_seq
    C_readcount = ComBat_seq(counts = as.matrix(U_readcount[,sample_names]), batch = batches, group = groups)

    #join the gene and chromosome names onto the now corrected counts from ComBat_seq
    C_readcount = cbind(U_readcount[,c("Gene","Chr")], C_readcount)

    #calculate principal components for the uncorrected data
    pca_corrected_obj = prcomp(C_readcount[,sample_names])

    #pull PCA values out of the PCA object
    pca_corrected = as.data.frame(pca_corrected_obj[2]$rotation)

    #assign labels to the data frame
    pca_corrected[,"Age_group"] = Age_group
    pca_corrected[,"Year_sampling"] = Year_sampling
    pca_corrected[,"replicate"] = replicates

    #as above, create a PCA plot for comparison to the uncorrected data
    p2 = ggplot(data=pca_corrected, aes(x=PC1, y=PC2, color=Age_group, shape=Year_sampling))
    p2 = p2 + stat_ellipse(type="norm", linetype=2)
    p2 = p2 + theme_bw() + theme(axis.title = element_text(size=20),panel.border = element_rect(size = 2, fill = NA),axis.text.y= element_text(size = 15),axis.text.x= element_text(size = 15))+
    geom_point(aes(color=Age_group), size =5)

    print(i)
    grid.arrange(p1,p2, nrow = 2)


    C_readcount <- C_readcount[,c(1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26)]
    
    names(C_readcount) = c("Gene",
                           'A_FST','B_FST','A_SND','B_SND','A_TRD','B_TRD',
                           'C_FST','D_FST','C_SND','D_SND','C_TRD','D_TRD',
                           'E_FST','F_FST','E_SND','F_SND','E_TRD','F_TRD',
                           'G_FST','H_FST','G_SND','H_SND','G_TRD','H_TRD')
    
    Name <- ifelse(startsWith(i,'Gene'),'gene_chr',ifelse(startsWith(i,'TE'),'TE_ID','miRNA'))

    colnames(C_readcount)[1]<- Name

    write.table(C_readcount, file=paste0('./source_data/batch_correction/batch_cor_',i), sep='\t',col.names = T, row.names = F)
    }
  